<a href="https://colab.research.google.com/github/ricardoricrob76/codigocoleta/blob/master/DIAD_carga_incremental_(CDC)_no_Apache_Iceberg_usando_Python%2C_com_a_coleta_de_dados_no_HDFS_e_o_armazenamento_definitivo_em_um_storage_S3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para criar uma carga incremental (CDC) no Apache Iceberg usando Python, com a coleta de dados no HDFS e o armazenamento definitivo em um storage S3, você pode seguir os passos abaixo:

Instale as bibliotecas necessárias: **negrito**
Certifique-se de ter as bibliotecas necessárias instaladas. Você precisará do pacote pyarrow para manipular os dados no HDFS e do pacote s3fs para trabalhar com o S3. Você pode instalá-los usando o pip:



In [1]:
pip install pyarrow s3fs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0


In [5]:
!pip install iceberg.api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement iceberg.api (from versions: none)
ERROR: No matching distribution found for iceberg.api


Importe os módulos necessários:
Importe os módulos necessários para trabalhar com o Apache Iceberg, o HDFS e o S3:

In [ ]:
from pyarrow import fs
from pyarrow import dataset
import s3fs
import pyarrow.parquet as pq
#from iceberg.api import FileFormat, Schema
#from iceberg.hive import HiveCatalog


Configure as credenciais do S3:
Configure as credenciais do S3 para acessar o seu bucket:

In [ ]:
s3_access_key = 'YOUR_ACCESS_KEY'
s3_secret_key = 'YOUR_SECRET_KEY'
s3_bucket = 'YOUR_S3_BUCKET'


Crie a tabela Iceberg:
Utilize a biblioteca HiveCatalog para criar a tabela Iceberg no HDFS. Certifique-se de fornecer o caminho HDFS desejado e o nome da tabela:

In [ ]:
hdfs_path = 'hdfs://your_hdfs_path'
table_name = 'your_table_name'

catalog = HiveCatalog.builder().with_hadoop_conf().build()
table = catalog.create_table(
    Schema([
        ('timestamp_col', 'timestamp'),
        ('data_col', 'string')
    ]),
    hdfs_path + '/' + table_name,
    format_=FileFormat.PARQUET
)


Certifique-se de fornecer o esquema correto para a tabela e configurar o caminho HDFS desejado.

Capture os dados incrementais no HDFS:
Utilize a biblioteca pyarrow para capturar os dados incrementais e gravá-los no HDFS. Aqui está um exemplo básico:

In [ ]:
staging_path = 'hdfs://your_staging_path'

# Capture os dados incrementais e salve-os em um arquivo Parquet
# Você precisa implementar a lógica de captura dos dados incrementais de acordo com a sua fonte de dados
incremental_data = [
    {'timestamp_col': '2023-01-01 10:00:00', 'data_col': 'Incremental data 1'},
    {'timestamp_col': '2023-01-01 11:00:00', 'data_col': 'Incremental data 2'}
]
df = pd.DataFrame(incremental_data)
table_path = staging_path + '/' + table_name + '.parquet'
pq.write_table(pa.Table.from_pandas(df), table_path)


Certifique-se de adaptar a lógica de captura dos dados incrementais de acordo com a sua fonte de dados.

Realize a carga incremental na tabela Iceberg:
Utilize a biblioteca pyarrow para carregar os dados incrementais na tabela Iceberg:

In [ ]:
table.load_in_place([table_path])


Copie os dados para o S3:
Utilize a biblioteca s3fs para copiar os dados do HDFS para o S3:

In [ ]:
s3 = s3fs.S3FileSystem(key=s3_access_key, secret=s3_secret_key)
s3_table_path = 's3://' + s3_bucket + '/' + table_name
s3fs.du(s3_table_path)  # Verifique se o caminho S3 já existe
s3fs.cp(table_path, s3_table_path, s3=s3)  # Copie os dados do HDFS para o S3


Certifique-se de fornecer suas próprias credenciais do S3 e configurar o caminho S3 desejado.

Dessa forma, você pode criar uma carga incremental (CDC) no Apache Iceberg usando Python, coletando os dados no HDFS e armazenando-os de forma definitiva em um storage S3. Certifique-se de adaptar o código de acordo com suas necessidades específicas.